In [12]:
import numpy as np
import fasttext
import pandas as pd
from resources.tokTT import CommentTokenizer as CT
from resources.basicIO import InputOutput as IO
from resources.filterLang import FilterLanguage as FL
from active_expansion.fasttext_batch_avg import Expander
from sklearn.model_selection import train_test_split


### Preprocess Corpus and Random Sample

In [13]:
raw_corpus = IO.load_text('datasets/corpus.txt')
tokenized_corpus = CT.cleaned('datasets/corpus.txt')
IO.save_text('datasets/tokenized_corpus.txt',tokenized_corpus)

### Make Fasttext Unsupervised Model

In [14]:
model_2 = fasttext.train_unsupervised(input="datasets/tokenized_corpus.txt", lr=0.01, epoch=10, wordNgrams=2, dim=100)
model_2.save_model("models/ft_unsupervised_N_2.bin")


### Make dataframes

In [15]:
# corpus
df_dict = {'raw_comment': raw_corpus, 'tokenized_comment': tokenized_corpus}
df_corpus = pd.DataFrame(df_dict)
df_corpus.to_csv('datasets/corpus_data.csv', index=False)

In [16]:
# random sample
text = IO.load_csv_col('datasets/random_sample.csv', 'comment')
text_labels = IO.load_csv_col('datasets/random_sample.csv', 'label')
text_labels = list(map(str, map(int, text_labels)))
text_TK = [CT.tokenize(x) for x in text]

df_dict = {'raw_comment': text, 'tokenized_comment': text_TK, 'label': text_labels}
df_sample = pd.DataFrame(df_dict)
df_sample.to_csv('datasets/random_sample_data.csv', index=False)


ValueError: cannot convert float NaN to integer

In [ ]:
# seed set
seed_set_text = IO.load_text('datasets/seed_set.txt')
seed_set_labels = IO.load_text('datasets/seed_set_labels.txt')
seed_set_labels = list(map(int, seed_set_labels))
seed_set_TK = CT.cleaned('datasets/seed_set.txt')

df_dict = {'raw_comment': seed_set_text,
           'tokenized_comment': seed_set_TK, 'label': seed_set_labels}
df_seed = pd.DataFrame(df_dict)
df_seed.to_csv('datasets/seed_set_data.csv', index=False)


### Train test split

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(df_sample['tokenized_comment'], 
                                                    df_sample['label'], test_size=0.1, 
                                                    random_state=42,
                                                    stratify=df_sample['label'])

### Use Active Learning on Training Data

### Seed Set Expansion